In [1]:
%matplotlib inline

import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
import matplotlib.pyplot as plt
import glob
import os

In [2]:
# Plot the histogram of the background (as you did in the Astropy1 tutorial). Do you see any changes? Can you track down in which step the change came about?
# There are a lot more aspects to image reduction than the steps here. Read up about these steps (which include cosmic ray removal, astrometry and possibly stacking) in the resources given below.
# The main purpose of the GROWTH-India Telescope is to find transients. These can be anything from Supernova (and other magnitudes of Novae) to afterglows of Gamma Ray Bursts, to Electromagnetic Counterparts of Gravitational Wave Sources. To keep in line with this, let us try to use a technique known as Image Subtraction, albeit with some simplifications (OK, that was a lie. Lots of simplification). Read more below

In [ ]:
# This returns the current path of the folder we are in
curpath = os.path.abspath('.')                  
# If you have extracted correctly, there should be a sub-folder named Bias, and you should get 3 files in the next cell
biasFolder = os.path.join(curpath, 'bias')
biasList = glob.glob(os.path.join(biasFolder,'*fits'))

numBiasFiles = len(biasList)
print('Found %d bias files'%numBiasFiles)
biasImages = np.zeros((4108, 4096, numBiasFiles))

for i in range(numBiasFiles):
        biasImages[:,:,i] = fits.open(biasList[i])[0].data
# This is to get a good scale on the plot, nothing more
mean, median, std = sigma_clipped_stats(biasImages[:,:,0])

plt.figure(figsize=(8,8))
plt.imshow(biasImages[:,:,0], vmin = median - 3*std, vmax = median + 3*std)
plt.colorbar()
masterBias = np.median(biasImages, axis=2)

mean, median, std = sigma_clipped_stats(masterBias)

plt.figure(figsize=(8,8))
plt.imshow(masterBias, vmin = median - 3*std, vmax = median + 3*std)
plt.colorbar()

Flat = fits.open('Flat.fits')[1].data

masterFlat = Flat - masterBias
master_flat_median = np.median(masterFlat)

# This re-scaling is so that we get a reasonable range in the final processed image. 
masterFlat = masterFlat/master_flat_median


mean, median, std = sigma_clipped_stats(masterFlat)
plt.figure(figsize=(8,8))
plt.imshow(masterFlat, vmin = median - 3*std, vmax = median + 3*std)
plt.colorbar()

rawHDU = fits.open('Messier3_raw.fits')[0]
rawData = rawHDU.data

# Let us have a look at the image:
plt.figure(figsize = (8,8))
mean, median, std = sigma_clipped_stats(rawData)
plt.imshow(rawData, vmin = median - 3*std, vmax = median + 3*std)
plt.colorbar()


rawHeader = rawHDU.header

procData = (rawData - masterBias) / masterFlat

procHDU = fits.PrimaryHDU(procData)
procHDU.header = rawHeader

procHDU.writeto('Messier3.proc.fits', overwrite=True)
M3_HDU = fits.open('Messier3.proc.fits')[0]
M3_data = M3_HDU.data

plt.figure(figsize = (8,8))
mean, median, std = sigma_clipped_stats(M3_data)
plt.imshow(M3_data, vmin = median - 3*std, vmax = median + 3*std, origin = "lower")
plt.colorbar()

In [3]:
def get_background_histogram(array, min_count, max_count):
    """
        array is the numpy array that contains the counts for each pixel
        the bins for the histogram will be np.arange(min_count, max_count, 1)
    """
    flattened_array = array.flatten()
    num_pixels, bin_edges = np.histogram(flattened_array, bins=np.arange(min_count, max_count, 1))
    bins = 0.5*(bin_edges[1:] + bin_edges[:-1])
    return bins, num_pixels
#Lets look at the background counts for the master bias frame

mean, median, std = sigma_clipped_stats(masterBias)
bins, num_pixels = get_background_histogram(masterBias,median - 5*std, median + 5*std) #background counts, not including cosmic ray counts
plt.figure(figsize = (10,6))
plt.title("Background counts, not including cosmic ray counts, for master bias frame")
plt.xlabel('Counts')
plt.ylabel('Number of Pixels')
plt.plot(bins, num_pixels)
plt.show()

NameError: name 'masterBias' is not defined

In [ ]:
#Lets look at the background counts for the master flats frame

my_masterFlat = masterFlat*master_flat_median #rescaling the master flat by multiplying by the median
                                              #master_flat_median is the median of the original master flat, the one before rescaling
                                              #the variable master_flat_median is defined in the cell just below the heading 'Creating a Master Flat frame'  

mean, median, std = sigma_clipped_stats(my_masterFlat)
bins, num_pixels = get_background_histogram(my_masterFlat,mean-5*std,mean+5*std) #background counts, not including cosmic ray counts
plt.figure(figsize = (10,6))
plt.title("Background counts, not including cosmic ray counts, for master flat frame")
plt.xlabel('Counts')
plt.ylabel('Number of Pixels')
plt.plot(bins, num_pixels)
plt.show()